### Thinking 1 排序模型按照样本生成方法和损失函数的不同，可以划分成Pointwise, Pairwise, Listwise三类方法，这三类排序方法有何区别



#### <font color=Red>Pointwise</font>

##### Pointwise方法主要针对单一文档，每个文档为单独训练数据。优点是算法简单。缺点是从单文档分类角度计算，忽略了文档之间的相对顺序，无法对文档进行排序。

##### 实际应用中，通常将训练样本转换为多分类问题（样本特征-类别标记）或者回归问题（样本特征-连续值）。

##### 如果转换为多分类问题，模型最后的输出只能1、3或是7。在同一类中的文档不好排序。

##### 如果转换为回归问题，常采用LR、GBDT等来解决。

#### <font color=Red>Pairwise</font>

##### Pairwise方法关注文档的顺序关系。优点是只需要对所有文档对进行分类，得到文档集的偏序关系。缺点是只考虑每对文档之间的偏序关系，实际上，文档对并不是相互独立的，而且不同的查询拥有的文档对数目不同，结果会向拥有文档对较多的查询偏移。

##### Pairwise没有考虑文档出现在搜索列表中的位置。因为排在搜索结果前面的文档更为重要，所以使用Pairwise方法对相对靠前的文档判断错误，代价会很高。

#### <font color=Red>Listwise</font>

##### Listwise方法将一次Query对应的所有搜索结果列表作为一个训练样例。优点是根据训练样例训练得到最优评分函数F，对应新的查询，评分F对每个文档打分，然后根据得分由高到低排序，得到最终的排序结果。不过，缺点是很难找到合适的目标来代替原有的优化目标，很难找到合适的优化算法进行求解。

##### 相对前两个方法来说，Listwise方法 直接考虑整体序列，针对Ranking评价指标（比如MAP, NDCG）进行优化。

##### Listwise相对其他方法来说计算量比较大，适合在搜索排序场景中（对排序精度要求较高的场景中）发挥作用。而在推荐系统场景中也可以用Listwise，但是根据推荐系统的特性来看，Listwise计算成本太高了，使用Pointwise就能得到一个较为不错的结果。而Pairwise相对Pointwise来说，多了先后顺序关系。但是相对Listwise来说，不能表示出位置关系。

## Thinking2 排序模型按照结构划分，可以分为线性排序模型、树模型、深度学习模型，这些模型都有哪些典型的代表？



#### <font color=Red>线性排序模型</font>

##### 线性模型典型代表是LR，引入自动二阶交叉特征的FM和FFM。LR模型可解释性好，方便问题定位和查找，通过特征权重可以解释feed排序的得分，可以人工分析和判断模型参数是否合理。

#### <font color=Red>树模型</font>

##### 线性模型典型代表是GBDT+LR。使用GBDT构造新特征，输出的并不是最终的二分类概率值，而是要把模型中的每棵树计算得到的预测概率值所属的叶子结点位置记为1。从而得到0和1组成的特征向量，再使用LR算法。这样可以得到比较好的结果。

#### <font color=Red>深度学习模型</font>

##### 深度学习模型代表有Wide & Deep, DeepFM, NFM。这几个算法，大体上比较类似，都是深度学习模型。输入层对特征进行embedding。使用LR或者FM处理一阶二阶特征，使用深度神经网络处理高阶特征。在处理顺序上略有些不同，DeepFM并行处理，NFM串行处理。

## Thinking3 NDCG如何计算


#### <font color=Red>CG</font>

##### CG即累计增益，只考虑到了相关性，没有考虑到位置的因素。直接将结果简单粗暴相加，没有考虑位置靠前或靠后的重要性不同。

#### <font color=Red>DCG</font>

##### DCG即折损累计增益，在CG的基础上，对每一个CG的结果上除以一个折损值，为了让排名靠前的结果能影响最后的结果。

##### 排序越往后价值越低。到第i个位置的时候，价值是 1/log2(i+1)，那么第i个结果产生的效益就是 reli * 1/log2(i+1)

#### <font color=Red>IDCG</font>

##### 理想情况下最大的DCG值。即结果按照相关性从大到小的顺序排序，取前p个结果组成的集合。也就是按照最优的方式对结果进行排序。

#### <font color=Red>NDCG</font>

##### NDCG，归一化贴现累积收益，综合考虑模型排序结果和真实序列之间的关系，最终使用的排序指标。因为DCG是一个累加的值，没法针对两个不同的搜索结果进行比较，需要归一化处理，即NDCG。


##### 因此 NDCG = DCG/IDCG

## Thinking4 搜索排序和推荐系统的相同和不同之处有哪些



#### <font color=Red>相同之处</font>

##### 两者对于LTR的依赖较高，一般来说，都离不开排序学习。排序的结果对推荐系统和搜索排序都很重要。

##### 两者都是一种推荐的过程，只不过推荐系统可以认为对用户来说是完全被动的推荐，而搜索排序是根据用户搜索内容对用户进行有意识的被动推荐。

##### 都可以根据用户特征，进行一些个性化的排序。

##### <font color=Red>不同之处</font>

##### 应用场景：

推荐系统，基于历史行为的“猜你喜欢”。搜索排序，基于某Query进行的结果排序，期望用户选中的在排序结果中是靠前的。

##### 方法方面：

推荐系统，采用pointwise模型较多，预测出来的分数，具有实际的物理意义，代表了目标用户点击item的预测概率。

搜索排序，采用pairwise和Listwise模型较多，一般排序靠前的item更重要。

##### 排序准确性方面：

搜索排序中排序的准确性是衡量这个搜索引擎最重要的指标之一。因此搜索排序对于排序准确性要求非常高。

推荐是发散的、无意识的主动推荐，相比search而言，排序准确性不一定是最重要的。多样性也导致了推荐场景没有像搜索一样适合做pairwise的样本。


个人理解（比如说我买了一个冰箱，因此推荐系统计算排序准确率的时候，冰箱一定排序结果较高，但是推荐系统不一定非得给我推荐冰箱，因为我短时间内再次买冰箱的概率很小。这时推荐系统准确率要求就没那么高，可以推荐一些其他的东西，比如活性炭。然而当我搜索冰箱的时候，搜索排序一定要注意准确性，因为我搜索冰箱很有可能会买冰箱。）

## Thinking5 Listwise排序模型能否应用到推荐系统中



##### Listwise是可以应用到推荐系统和搜索排序中的。

##### Listwise将一次Query对应的所有搜索结果列表作为一个训练样例，计算量相比Pointwise高，排序准确性相比Pointwise也较高。但是在推荐系统中不是排序准确性高就一定好，推荐系统中多样性很重要。就像上一个问题举的冰箱的例子一样。

Pointwise方法对于推荐系统来说，推荐排序的效果已经比较好了，Listwise虽然能够提升一点点效果，但是代价比较大，而且有些时候可能会适得其反。杀鸡焉用宰牛刀，还是搜索排序更适合Listwise方法发挥作用。

## Action1 数据集：porto seguro safe driver prediction（kaggle 2017年比赛）
Porto Seguro是巴西最大的汽车与住房保险公司之一，根据汽车保单持有人的数据建立机器学习模型，分析该持有人是否会在次年提出索赔。数据已进行脱敏
使用GBDT+LR模型进行预测，并计算Normalized Cross Entropy


##### 详情请见 GBDT_LR.ipynb 文件

## Action2 熟悉RankLib的使用，针对MQ2008数据集中的Fold1，使用RankNet, ListNet, LambdaMart三种模型进行排序学习，并对比三种模型的NDCG@10结果



##### 使用RankLib中的RankLib-patched工具进行排序学习。

##### <font color=Red>很重要的一点，使用jar工具需要在命令行中切换到jar工具的路径。才可以正确使用。
    
如 cd F:\kkb\RS\homework\L10</font>

#### <font color=Red>RankNet</font>

##### 执行的命令

java -jar RankLib-patched.jar -train MQ2008/Fold1/train.txt -test MQ2008/Fold1/test.txt -validate MQ2008/Fold1/vali.txt -ranker <font color=Red>1</font> -metric2t NDCG@10 -metric2T NDCG@8 -save <font color=Red>RankNet_xueliangchen.txt</font>

##### ![Ranknet_xueliangchen.txt文件中部分信息](1.png)

##### 使用RankNet方法，迭代了100次，得到下面的结果

##### ![Ranknet结果](2.png)

#### <font color=Red>ListNet</font>

##### 执行的命令

java -jar RankLib-patched.jar -train MQ2008/Fold1/train.txt -test MQ2008/Fold1/test.txt -validate MQ2008/Fold1/vali.txt -ranker <font color=Red>7</font> -metric2t NDCG@10 -metric2T NDCG@8 -save <font color=Red>ListNet_xueliangchen.txt</font>

##### ![ListNet_xueliangchen.txt文件中部分信息](3.png)

##### 使用ListNet方法，迭代了1500次，得到下面的结果

##### ![ListNet结果](4.png)

#### <font color=Red>LambdaMart</font>

##### 执行的命令

java -jar RankLib-patched.jar -train MQ2008/Fold1/train.txt -test MQ2008/Fold1/test.txt -validate MQ2008/Fold1/vali.txt -ranker <font color=Red>6 </font> -metric2t NDCG@10 -metric2T NDCG@8 -save <font color=Red>LambdaMart_xueliangchen.txt</font>

##### ![LambdaMart_xueliangchen.txt文件中部分信息](5.png)

##### 使用ListNet方法，迭代了135次，得到下面的结果

##### ![LambdaMart结果](6.png)

#### <font color=Red>多个模型进行对比</font>

##### 生成baseline NDCG数据

##### ![生成baseline NDCG数据](7.png)

##### 分别训练三个模型（RankNet, ListNet, LambdaMart）

<font color=Orange>RankNet</font>

java -jar RankLib-patched.jar -train MQ2008/Fold1/train.txt -test MQ2008/Fold1/test.txt -validate MQ2008/Fold1/vali.txt -ranker 1 -metric2t NDCG@10 -metric2T NDCG@10 -save <font color=Red>model_ranknet.txt</font>

<font color=Orange>ListNet</font>

java -jar RankLib-patched.jar -train MQ2008/Fold1/train.txt -test MQ2008/Fold1/test.txt -validate MQ2008/Fold1/vali.txt -ranker 7 -metric2t NDCG@10 -metric2T NDCG@10 -save <font color=Red>model_listnet.txt</font>

<font color=Orange>LambdaMart</font>

java -jar RankLib-patched.jar -train MQ2008/Fold1/train.txt -test MQ2008/Fold1/test.txt -validate MQ2008/Fold1/vali.txt -ranker 6 -metric2t NDCG@10 -metric2T NDCG@10 -save <font color=Red>model_lambdamart.txt</font>

##### 得出三个模型的NDCG结果（RankNet, ListNet, LambdaMart）

<font color=Orange>RankNet</font>

java -jar RankLib-patched.jar -load model_ranknet.txt -test MQ2008/Fold1/test.txt -metric2T NDCG@10 -idv output/<font color=Red>ranknet.ndcg.txt</font>

![生成ranknet.ndcg.txt文件](8.png)

<font color=Orange>ListNet</font>

java -jar RankLib-patched.jar -load model_listnet.txt -test MQ2008/Fold1/test.txt -metric2T NDCG@10 -idv output/<font color=Red>listnet.ndcg.txt</font>

![生成listnet.ndcg.txt文件](9.png)

<font color=Orange>LambdaMart</font>

java -jar RankLib-patched.jar -load model_lambdamart.txt -test MQ2008/Fold1/test.txt -metric2T NDCG@10 -idv output/<font color=Red>lambdamart.ndcg.txt</font>

![生成lambdamart.ndcg.txt文件](10.png)

##### 多个模型对比（RankNet, ListNet, LambdaMart）

##### 使用如下命令，将对比结果输出到analysis.txt文件中

java -cp RankLib-patched.jar ciir.umass.edu.eval.Analyzer -all output/ -base baseline.ndcg.txt > analysis.txt

##### ![模型对比结果](11.png)

## Action3 如果你是某P2P租车的技术负责人，你会如何设计个性化推荐和搜索排序，阐述相似车型，搜索排序的设计方法，可能的embedding策略



### 分析

<font color=Red>p2p租车业务模式：</font>

私家车主在他车辆闲置的时候，可以把他的车上传至平台，出租给需要用车的人；需要用车的人可以通过平台迅速寻找到附近的车。资源可以得到合理分配。

<font color=Red>p2p租车优点：</font>

一是说服消费者享受出行之乐不必自购车辆，这一点诸如神州租车这样的先驱已经进行了多年的市场教育；二是告诉车主闲置车辆不是负担而是摇钱树，有人以为这就是出行版的Airbnb

细来看，P2P租车和Airbnb的业务类似。因此可以借鉴Airbnb个性化推荐以及搜索排序的设计方法，embedding策略。

##### <font color=Red>p2p租车</font>

短租：车是分时段的，只能在某段时间内被某一个用户预订。

##### 租车是低频事件（对于大部分预定用户）

##### p2p租车也是一个双边平台，不仅顾客可以选择车的类型，车主也可以选择是否租给某个顾客

### p2p个性化推荐

##### 针对搜索排序，相似车型推荐进行的实时个性化

##### 对于query（带有用车位置和时间），同时为host和guest优化搜索结果：

顾客角度：需要根据位置，价格，类型，评论等因素排序来获得客户喜欢的listing

房东角度：需要过滤掉那些有坏的评论，宠物，不良习惯等其他因素而拒绝guest的listing，将这些listing排列的低一点

和Airbnb一样，采用 <font color=Red>Learnig to rank</font>来做，将问题转换为<font color=Red>pairwise regression</font>问题，将预定的listing作为正样本，拒绝的作为负样本。

##### <font color=Red>房源embedding：</font>

把每个用户连续点击过的房源Session看做一个句子，每个房源当做word，训练出房源的embedding。

##### 在原始Skip-Gram构造的目标函数基础上，使用负采样，用户的预定行为，适配聚集搜索等策略优化得到最终的目标函数

##### <font color=Red>通过embedding方法将车源转化成同一个空间上的向量，可以通过比较向量的余弦相似度等指标，可以很方便的得到相似车型————通过在list embedding空间中找到 k个最近邻居</font>

##### 可以使用embedding解决冷启动问题。在和新上传车源具有相同类型和相同价格区间的车源中，找到3个地理位置最接近的车源，用这3个车源的embedding求平均作为新车源的embedding。很好的解决了冷启动问题。


##### Listing Embedding在实时推荐效果不错，但是不适合长期。

长期 可以借鉴Airbnb中的User Type Embedding 和Listing Type Embedding策略


##### 基于Embedding的搜索排序：GBDT模型（支持lambda rank）解决pairwise问题（3种embedding值，Listing Embedding，User Type Embedding和Listing Type Embedding）